In [1]:
library(tidyverse)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 1

In [4]:
p_thresh = 0.05
lfc_thresh = log2(2)
coxph_coeff_thresh = 0.0

# Helpers

In [5]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list) %>%
    dplyr::select(gene_symbol, division, category)

Parsed with column specification:
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)


In [6]:
norm_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/", "norm_counts.tsv"))

Parsed with column specification:
cols(
  .default = col_double(),
  geneID = col_character()
)
See spec(...) for full column specifications.


# DGE analysis

In [7]:
DESeq_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_DESeq_results.tsv"))
filtered_DESeq_results_df <- DESeq_results_df %>%
    dplyr::filter(abs(log2FoldChange) > lfc_thresh, padj < p_thresh)
filtered_matrisome_DESeq_results_df <- filtered_DESeq_results_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)

n_degs <- nrow(filtered_DESeq_results_df)
deg_prop <- nrow(filtered_DESeq_results_df) / nrow(norm_counts_df)
n_degs_up <- nrow(filtered_DESeq_results_df %>% dplyr::filter(log2FoldChange > 0))
n_degs_down <- nrow(filtered_DESeq_results_df %>% dplyr::filter(log2FoldChange < 0))

n_matrisome_degs <- nrow(filtered_matrisome_DESeq_results_df)
matrisome_deg_prop <- nrow(filtered_matrisome_DESeq_results_df) / nrow(matrisome_df)
n_matrisome_degs_up <- nrow(filtered_matrisome_DESeq_results_df %>% dplyr::filter(log2FoldChange > 0))
n_matrisome_degs_down <- nrow(filtered_matrisome_DESeq_results_df %>% dplyr::filter(log2FoldChange < 0))


paste0("DEGs: ", n_degs)
paste0("DEG prop: ", deg_prop)
paste0("Up: ", n_degs_up)
paste0("Down: ", n_degs_down)
paste0("Matrisome DEGs: ", n_matrisome_degs)
paste0("Matrisome DEG prop: ", matrisome_deg_prop)
paste0("Matrisome up: ", n_matrisome_degs_up)
paste0("Matrisome down: ", n_matrisome_degs_down)

Parsed with column specification:
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


[1] "DEGs: 7189"

[1] "DEG prop: 0.355152652899911"

[1] "Up: 3703"

[1] "Down: 3486"

[1] "Matrisome DEGs: 559"

[1] "Matrisome DEG prop: 0.544303797468354"

[1] "Matrisome up: 241"

[1] "Matrisome down: 318"

# Survival analysis

In [8]:
coxph_null_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "coxph_null_scores.tsv"))
coxph_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_coxph_results.tsv"))
filtered_coxph_results_df <- coxph_results_df %>%
    dplyr::filter(gene_pval < p_thresh)

Parsed with column specification:
cols(
  dataset = col_character(),
  lr_test_pval = col_double(),
  wald_test_pval = col_double(),
  score_test_pval = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  gene_pval = col_double(),
  gene_coeff = col_double()
)


In [9]:
coxph_null_scores_df

dataset,lr_test_pval,wald_test_pval,score_test_pval
<chr>,<dbl>,<dbl>,<dbl>
unified_cervical_data,0.0004814013,0.000049053,1.224173e-06
unified_uterine_data,0.3392807906,0.989893132,3.053979e-01
unified_uterine_endometrial_data,0.0007739010,0.000000000,1.040876e-06


In [10]:
coxph_null_sig <- (coxph_null_scores_df %>% dplyr::filter(dataset == unified_dsets[dset_idx]))$lr_test_pval < p_thresh
n_coxph_sig <- nrow(filtered_coxph_results_df)
prop_coxph_sig <- n_coxph_sig / nrow(matrisome_df)
n_coxph_sig_protective <- nrow(filtered_coxph_results_df %>% dplyr::filter(gene_coeff < 0))
n_coxph_sig_harmful <- nrow(filtered_coxph_results_df %>% dplyr::filter(gene_coeff > 0))


paste0("Sig. null Cox PH: ", coxph_null_sig)
paste0("Sig. Cox PH genes: ", n_coxph_sig)
paste0("Prop. sig. Cox PH genes: ", prop_coxph_sig)
paste0("Sig. Cox PH protective: ", n_coxph_sig_protective)
paste0("Sig. Cox PH harmful: ", n_coxph_sig_harmful)

[1] "Sig. null Cox PH: TRUE"

[1] "Sig. Cox PH genes: 171"

[1] "Prop. sig. Cox PH genes: 0.166504381694255"

[1] "Sig. Cox PH protective: 54"

[1] "Sig. Cox PH harmful: 117"

# Regression

## Correlation analysis

In [11]:
cor_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_cor_results.tsv"))

filtered_cor_results_df <- cor_results_df %>% dplyr::filter(pval < p_thresh)

n_cor <- nrow(filtered_cor_results_df)
n_cor_down <- nrow(filtered_cor_results_df %>% dplyr::filter(cor < 0))
n_cor_up <- nrow(filtered_cor_results_df %>% dplyr::filter(cor > 0))

paste0("Cor. genes: ", n_cor)
paste0("Neg. cor. genes: ", n_cor_down)
paste0("Pos. cor. genes: ", n_cor_up)

Parsed with column specification:
cols(
  geneID = col_character(),
  cor = col_double(),
  pval = col_double(),
  n = col_double()
)


[1] "Cor. genes: 85"

[1] "Neg. cor. genes: 33"

[1] "Pos. cor. genes: 52"

## MI analysis

In [12]:
mi_survival_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_MI_survival_results.tsv"))
filtered_mi_survival_results_df <- mi_survival_results_df %>% dplyr::filter(MI_est_median > 0)

n_mi <- nrow(filtered_mi_survival_results_df)

paste0("MI: ", n_mi)

Parsed with column specification:
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)


[1] "MI: 550"

## Baselines

In [13]:
reg_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "reg_baselines.tsv"))
mae_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$L1
ev_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$explained_variance
paste0("MAE baseline: ", mae_baseline)
paste0("EV baseline: ", ev_baseline)

Parsed with column specification:
cols(
  dataset = col_character(),
  L2 = col_double(),
  L1 = col_double(),
  R2 = col_double(),
  explained_variance = col_double(),
  n = col_double()
)


[1] "MAE baseline: 518.333333333333"

[1] "EV baseline: 0"

## GBR (MAE)

In [14]:
mae_gbr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_mae_gbr_ref_scores.tsv"))
mae_gbr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_mae_gbr_results.tsv"))
mae_gbr_scores_df

mae_gbr_avg <- -mean(mae_gbr_scores_df$ref_score)
# Want MAE to be < baseline
mae_gbr_imp <- mae_gbr_avg < mae_baseline
mae_gbr_baseline_diff <- mae_gbr_avg - mae_baseline
n_mae_gbr_consensus_genes <- nrow(mae_gbr_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("GBR MAE avg.: ", mae_gbr_avg)
paste0("GBR MAE impr. over baseline: ", mae_gbr_imp)
paste0("GBR MAE diff with baseline: ", mae_gbr_baseline_diff)
paste0("GBR MAE consensus genes: ", n_mae_gbr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,-518.8947
1,-512.7294
2,-507.1236
3,-507.3762
4,-511.5485


[1] "GBR MAE avg.: 511.53448699008"

[1] "GBR MAE impr. over baseline: TRUE"

[1] "GBR MAE diff with baseline: -6.79884634325339"

[1] "GBR MAE consensus genes: 50"

## GBR (EV)

In [15]:
ev_gbr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_ev_gbr_ref_scores.tsv"))
ev_gbr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_ev_gbr_results.tsv"))
ev_gbr_scores_df

ev_gbr_avg <- mean(ev_gbr_scores_df$ref_score)
# Want EV to be > baseline
ev_gbr_imp <- ev_gbr_avg > ev_baseline
ev_gbr_baseline_diff <- ev_gbr_avg - ev_baseline
n_ev_gbr_consensus_genes <- nrow(ev_gbr_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("GBR EV avg.: ", ev_gbr_avg)
paste0("GBR EV impr. over baseline: ", ev_gbr_imp)
paste0("GBR EV diff with baseline: ", ev_gbr_baseline_diff)
paste0("GBR EV consensus genes: ", n_ev_gbr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,-0.22036053
1,-0.18572014
2,0.01803834
3,-0.03090779
4,0.02025601


[1] "GBR EV avg.: -0.0797388205342726"

[1] "GBR EV impr. over baseline: FALSE"

[1] "GBR EV diff with baseline: -0.0797388205342726"

[1] "GBR EV consensus genes: 46"

## RFR (MAE)

In [16]:
mae_rfr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_mae_rfr_ref_scores.tsv"))
mae_rfr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_mae_rfr_results.tsv"))
mae_rfr_scores_df

mae_rfr_avg <- -mean(mae_rfr_scores_df$ref_score)
# Want MAE to be < baseline
mae_rfr_imp <- mae_rfr_avg < mae_baseline
mae_rfr_baseline_diff <- mae_rfr_avg - mae_baseline
n_mae_rfr_consensus_genes <- nrow(mae_rfr_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("RFR MAE avg.: ", mae_rfr_avg)
paste0("RFR MAE impr. over baseline: ", mae_rfr_imp)
paste0("RFR MAE diff with baseline: ", mae_rfr_baseline_diff)
paste0("RFR MAE consensus genes: ", n_mae_rfr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,-565.1224
1,-574.7631
2,-578.7912
3,-568.5393
4,-580.6566


[1] "RFR MAE avg.: 573.574538465928"

[1] "RFR MAE impr. over baseline: FALSE"

[1] "RFR MAE diff with baseline: 55.2412051325946"

[1] "RFR MAE consensus genes: 55"

## RFR (EV)

In [17]:
ev_rfr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_ev_rfr_ref_scores.tsv"))
ev_rfr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_ev_rfr_results.tsv"))
ev_rfr_scores_df

ev_rfr_avg <- mean(ev_rfr_scores_df$ref_score)
# Want EV to be > baseline
ev_rfr_imp <- ev_rfr_avg > ev_baseline
ev_rfr_baseline_diff <- ev_rfr_avg - ev_baseline
n_ev_rfr_consensus_genes <- nrow(ev_rfr_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("RFR EV avg.: ", ev_rfr_avg)
paste0("RFR EV impr. over baseline: ", ev_rfr_imp)
paste0("RFR EV diff with baseline: ", ev_rfr_baseline_diff)
paste0("RFR EV consensus genes: ", n_ev_rfr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,-0.010738678
1,0.005390867
2,0.015761948
3,0.020397702
4,0.005942691


[1] "RFR EV avg.: 0.00735090603733708"

[1] "RFR EV impr. over baseline: TRUE"

[1] "RFR EV diff with baseline: 0.00735090603733708"

[1] "RFR EV consensus genes: 93"

# Classification

## MI analysis

In [18]:
mi_figo_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_MI_figo_results.tsv"))
filtered_mi_figo_results_df <- mi_figo_results_df %>% dplyr::filter(MI_est_median > 0)

n_mi <- nrow(filtered_mi_figo_results_df)

paste0("MI: ", n_mi)

Parsed with column specification:
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)


[1] "MI: 506"

## Baselines

In [19]:
cls_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "cls_baselines.tsv"))
cls_baselines_df
f1_weighted_majority_baseline <- (cls_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$f1_weighted_majority
f1_weighted_MC_baseline <- (cls_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$f1_weighted_MC


paste0("F1 weighted (majority guess) baseline: ", f1_weighted_majority_baseline)
paste0("F1 weighted (Monte Carlo) baseline: ", f1_weighted_MC_baseline)

Parsed with column specification:
cols(
  dataset = col_character(),
  f1_weighted_majority = col_double(),
  f1_weighted_MC = col_double(),
  n = col_double()
)


dataset,f1_weighted_majority,f1_weighted_MC,n
<chr>,<dbl>,<dbl>,<dbl>
unified_cervical_data,0.3665158,0.3679826,255
unified_uterine_data,0.2201087,0.3113742,46
unified_uterine_endometrial_data,0.3704553,0.3814409,137


[1] "F1 weighted (majority guess) baseline: 0.366515837104072"

[1] "F1 weighted (Monte Carlo) baseline: 0.367982618790009"

## GBC

In [20]:
f1_gbc_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_gbc_ref_scores.tsv"))
f1_gbc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_gbc_results.tsv"))
f1_gbc_scores_df

f1_gbc_avg <- mean(f1_gbc_scores_df$ref_score)
f1_gbc_imp <- f1_gbc_avg > f1_weighted_majority_baseline
f1_gbc_baseline_diff <- f1_gbc_avg - f1_weighted_majority_baseline
n_f1_gbc_consensus_genes <- nrow(f1_gbc_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("GBC F1 avg.: ", f1_gbc_avg)
paste0("GBC F1 impr. over baseline: ", f1_gbc_imp)
paste0("GBC F1 diff with baseline: ", f1_gbc_baseline_diff)
paste0("GBC F1 consensus genes: ", n_f1_gbc_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,0.4972759
1,0.4947159
2,0.4847261
3,0.4839175
4,0.4890497


[1] "GBC F1 avg.: 0.489937009224344"

[1] "GBC F1 impr. over baseline: TRUE"

[1] "GBC F1 diff with baseline: 0.123421172120272"

[1] "GBC F1 consensus genes: 25"

## RFC

In [21]:
f1_rfc_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_rfc_ref_scores.tsv"))
f1_rfc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_rfc_results.tsv"))
f1_rfc_scores_df

f1_rfc_avg <- mean(f1_rfc_scores_df$ref_score)
f1_rfc_imp <- f1_rfc_avg > f1_weighted_majority_baseline
f1_rfc_baseline_diff <- f1_rfc_avg - f1_weighted_majority_baseline
n_f1_rfc_consensus_genes <- nrow(f1_rfc_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("RBC F1 avg.: ", f1_rfc_avg)
paste0("RBC F1 impr. over baseline: ", f1_rfc_imp)
paste0("RBC F1 diff with baseline: ", f1_rfc_baseline_diff)
paste0("RBC F1 consensus genes: ", n_f1_rfc_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,0.3916562
1,0.4086013
2,0.3904341
3,0.3926176
4,0.3970711


[1] "RBC F1 avg.: 0.396076075381531"

[1] "RBC F1 impr. over baseline: TRUE"

[1] "RBC F1 diff with baseline: 0.0295602382774589"

[1] "RBC F1 consensus genes: 0"

## LR (L1)

In [22]:
f1_l1_lr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l1_lr_ref_scores.tsv"))
f1_l1_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l1_lr_results.tsv"))
f1_l1_lr_scores_df

f1_l1_lr_avg <- mean(f1_l1_lr_scores_df$ref_score)
f1_l1_lr_imp <- f1_l1_lr_avg > f1_weighted_majority_baseline
f1_l1_lr_baseline_diff <- f1_l1_lr_avg - f1_weighted_majority_baseline
n_f1_l1_lr_consensus_genes <- nrow(f1_l1_lr_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("LR L1 F1 avg.: ", f1_l1_lr_avg)
paste0("LR L1 F1 impr. over baseline: ", f1_l1_lr_imp)
paste0("LR L1 F1 diff with baseline: ", f1_l1_lr_baseline_diff)
paste0("LR L1 F1 consensus genes: ", n_f1_l1_lr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,0.5061608
1,0.5089790
2,0.5091950
3,0.5091950
4,0.5016497


[1] "LR L1 F1 avg.: 0.507035926627848"

[1] "LR L1 F1 impr. over baseline: TRUE"

[1] "LR L1 F1 diff with baseline: 0.140520089523776"

[1] "LR L1 F1 consensus genes: 166"

## LR (L2)

In [23]:
f1_l2_lr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l2_lr_ref_scores.tsv"))
f1_l2_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l2_lr_results.tsv"))
f1_l2_lr_scores_df

f1_l2_lr_avg <- mean(f1_l2_lr_scores_df$ref_score)
f1_l2_lr_imp <- f1_l2_lr_avg > f1_weighted_majority_baseline
f1_l2_lr_baseline_diff <- f1_l2_lr_avg - f1_weighted_majority_baseline
n_f1_l2_lr_consensus_genes <- nrow(f1_l2_lr_results_df %>% dplyr::filter(consensus_vote == TRUE))

paste0("LR L2 F1 avg.: ", f1_l2_lr_avg)
paste0("LR L2 F1 impr. over baseline: ", f1_l2_lr_imp)
paste0("LR L2 F1 diff with baseline: ", f1_l2_lr_baseline_diff)
paste0("LR L2 F1 consensus genes: ", n_f1_l2_lr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


model,ref_score
<dbl>,<dbl>
0,0.5097446
1,0.5097446
2,0.5097446
3,0.5097446
4,0.5097446


[1] "LR L2 F1 avg.: 0.509744581003057"

[1] "LR L2 F1 impr. over baseline: TRUE"

[1] "LR L2 F1 diff with baseline: 0.143228743898984"

[1] "LR L2 F1 consensus genes: 869"